In [1]:
from datetime import datetime
from django.utils.timezone import utc

from music.choices import MUSIC_GENRES_CHOICES
from music.models import Album, Song, Artist, RecordCompany
from music.utils import *

In [2]:
clean_database()
create_albums()
create_artists_composers()
create_songs()

In [3]:
Artist.objects.filter(songs_written__genre=MUSIC_GENRES_CHOICES.pop_music).distinct()

<QuerySet [<Artist: Dua Lipa>, <Artist: Dave Bayley>, <Artist: Julia Michaels>, <Artist: Billie Eilish>, <Artist: Sarah Hudson>, <Artist: Finneas O'Connell>]>

In [4]:
# Count example

from django.db.models import Count

Artist.objects.annotate(Count("songs_written")).filter(songs_written__count__gt=4)

<QuerySet [<Artist: Finneas O'Connell>, <Artist: Billie Eilish>]>

In [5]:
# Count SQL query
print(Artist.objects.annotate(Count("songs_written")).filter(songs_written__count__gt=4).query)

SELECT "music_artist"."id", "music_artist"."name", "music_artist"."birthday", COUNT("music_song_composers"."song_id") AS "songs_written__count" FROM "music_artist" LEFT OUTER JOIN "music_song_composers" ON ("music_artist"."id" = "music_song_composers"."artist_id") GROUP BY "music_artist"."id" HAVING COUNT("music_song_composers"."song_id") > 4


In [6]:
# Using F with annotations

from django.db.models import F, Value
from django.db.models.functions import Concat

Album.objects.annotate(album_label=Concat(F("album_title"), Value(" by "), F("artist"))).all().first().__dict__

{'_state': <django.db.models.base.ModelState at 0x7f51f5ef0850>,
 'id': 68,
 'album_title': 'Future Nostalgia',
 'artist': 'Dua Lipa',
 'record_company_id': None,
 'release_date': datetime.datetime(2020, 3, 27, 3, 59, 0, 99999, tzinfo=<UTC>),
 'grammy_nominated': True,
 'album_label': 'Future Nostalgia by Dua Lipa'}

In [7]:
# Annotation + F SQL query
print(Album.objects.annotate(album_label=Concat(F("album_title"), Value(" by "), F("artist"))).all().query)

SELECT "music_album"."id", "music_album"."album_title", "music_album"."artist", "music_album"."record_company_id", "music_album"."release_date", "music_album"."grammy_nominated", CONCAT("music_album"."album_title", CONCAT( by , "music_album"."artist")) AS "album_label" FROM "music_album"


In [8]:
# Annotation with filters

from django.db.models import Q

Album.objects.annotate(
    electronic_songs_count=Count("songs", filter=Q(songs__genre=MUSIC_GENRES_CHOICES.rock))
).filter(electronic_songs_count__gt=1)

<QuerySet [<Album: Social Cues by Cage The Elephant>, <Album: The Colour and the Shape by Foo Fighters>, <Album: Dreamland by Glass Animals>]>

In [9]:
# Annotation with filters SQL query
print(Album.objects.annotate(
    electronic_songs_count=Count("songs", filter=Q(songs__genre=MUSIC_GENRES_CHOICES.rock))
).filter(electronic_songs_count__gt=1).query)

SELECT "music_album"."id", "music_album"."album_title", "music_album"."artist", "music_album"."record_company_id", "music_album"."release_date", "music_album"."grammy_nominated", COUNT("music_song"."id") FILTER (WHERE "music_song"."genre" = rock) AS "electronic_songs_count" FROM "music_album" LEFT OUTER JOIN "music_song" ON ("music_album"."id" = "music_song"."album_id") GROUP BY "music_album"."id" HAVING COUNT("music_song"."id") FILTER (WHERE ("music_song"."genre" = rock)) > 1


In [14]:
# Using FilteredRelation with annotations

from django.db.models import FilteredRelation, F, Q

# Filtering artists that have composed pop songs that were nominated to the Grammys
# Artist.objects.annotate(
#     grammy_nominated_songs=FilteredRelation(
#         "songs_written", condition=Q(songs_written__grammy_nominated=True)
#     )
# ).filter(grammy_nominated_songs__genre=MUSIC_GENRES_CHOICES.pop_music)

# Filtering Albums with rock songs that have the same title as the album title
Album.objects.annotate(
    same_title_songs=FilteredRelation(
        "songs", condition=Q(songs__song_title=F("album_title"))
    )
).filter(same_title_songs__genre=MUSIC_GENRES_CHOICES.rock)

<QuerySet [<Album: Social Cues by Cage The Elephant>]>

In [15]:
print(Album.objects.annotate(
    same_title_songs=FilteredRelation(
        "songs", condition=Q(songs__song_title=F("album_title"))
    )
).filter(same_title_songs__genre=MUSIC_GENRES_CHOICES.rock).query)

SELECT "music_album"."id", "music_album"."album_title", "music_album"."artist", "music_album"."record_company_id", "music_album"."release_date", "music_album"."grammy_nominated" FROM "music_album" INNER JOIN "music_song" same_title_songs ON ("music_album"."id" = same_title_songs."album_id" AND (same_title_songs."song_title" = "music_album"."album_title")) WHERE same_title_songs."genre" = rock


In [20]:
from django.db.models import FilteredRelation, F, Q

print(Artist.objects.annotate(
    grammy_nominated_songs=FilteredRelation(
        "songs_written", condition=Q(songs_written__grammy_nominated=True)
    )
).filter(grammy_nominated_songs__genre=MUSIC_GENRES_CHOICES.pop_music).explain(analyze=True))

Hash Join  (cost=2.46..3.83 rows=3 width=30) (actual time=0.084..0.093 rows=6 loops=1)
  Hash Cond: (music_song_composers.artist_id = music_artist.id)
  ->  Hash Join  (cost=1.24..2.60 rows=3 width=8) (actual time=0.041..0.048 rows=6 loops=1)
        Hash Cond: (music_song_composers.song_id = grammy_nominated_songs.id)
        ->  Seq Scan on music_song_composers  (cost=0.00..1.27 rows=27 width=16) (actual time=0.004..0.007 rows=27 loops=1)
        ->  Hash  (cost=1.21..1.21 rows=2 width=8) (actual time=0.015..0.016 rows=3 loops=1)
              Buckets: 1024  Batches: 1  Memory Usage: 9kB
              ->  Seq Scan on music_song grammy_nominated_songs  (cost=0.00..1.21 rows=2 width=8) (actual time=0.005..0.008 rows=3 loops=1)
                    Filter: (grammy_nominated AND ((genre)::text = 'pop_music'::text))
                    Rows Removed by Filter: 14
  ->  Hash  (cost=1.10..1.10 rows=10 width=30) (actual time=0.023..0.024 rows=10 loops=1)
        Buckets: 1024  Batches: 1  Memo

In [21]:
# print(Artist.objects.filter(songs_written__grammy_nominated=True, songs_written__genre=MUSIC_GENRES_CHOICES.pop_music).query)

print(Artist.objects.filter(songs_written__grammy_nominated=True, songs_written__genre=MUSIC_GENRES_CHOICES.pop_music).explain(analyze=True))

Hash Join  (cost=2.46..3.83 rows=3 width=30) (actual time=0.111..0.128 rows=6 loops=1)
  Hash Cond: (music_song_composers.artist_id = music_artist.id)
  ->  Hash Join  (cost=1.24..2.60 rows=3 width=8) (actual time=0.052..0.065 rows=6 loops=1)
        Hash Cond: (music_song_composers.song_id = music_song.id)
        ->  Seq Scan on music_song_composers  (cost=0.00..1.27 rows=27 width=16) (actual time=0.004..0.009 rows=27 loops=1)
        ->  Hash  (cost=1.21..1.21 rows=2 width=8) (actual time=0.028..0.029 rows=3 loops=1)
              Buckets: 1024  Batches: 1  Memory Usage: 9kB
              ->  Seq Scan on music_song  (cost=0.00..1.21 rows=2 width=8) (actual time=0.008..0.014 rows=3 loops=1)
                    Filter: (grammy_nominated AND ((genre)::text = 'pop_music'::text))
                    Rows Removed by Filter: 14
  ->  Hash  (cost=1.10..1.10 rows=10 width=30) (actual time=0.042..0.043 rows=10 loops=1)
        Buckets: 1024  Batches: 1  Memory Usage: 9kB
        ->  Seq Scan 

In [33]:
print(Artist.objects.annotate(
    grammy_nominated_songs=FilteredRelation(
        "songs_written", condition=Q(songs_written__grammy_nominated=True)
    )
).filter(grammy_nominated_songs__genre=MUSIC_GENRES_CHOICES.pop_music).query)

SELECT "music_artist"."id", "music_artist"."name", "music_artist"."birthday" FROM "music_artist" INNER JOIN "music_song_composers" ON ("music_artist"."id" = "music_song_composers"."artist_id") INNER JOIN "music_song" grammy_nominated_songs ON ("music_song_composers"."song_id" = grammy_nominated_songs."id" AND (grammy_nominated_songs."grammy_nominated")) WHERE grammy_nominated_songs."genre" = pop_music


In [27]:
# Using SubQuery with annotations

from django.db.models import OuterRef, Subquery

Artist.objects.annotate(latest_released_song_title=Subquery(
    Song.objects.filter(composers=OuterRef("id")).order_by("-release_date").values("song_title")[:1]
)).values_list("name", "latest_released_song_title")

<QuerySet [('Billie Eilish', 'Oxytocin'), ("Finneas O'Connell", 'Oxytocin'), ('Marina Diamandis', 'I Am Not A Robot'), ('Dave Bayley', 'I Dont Wanna Talk (I Just Wanna Dance)'), ('Brittany Hazzard', 'Tangerine'), ('Paul Epworth', 'Tangerine'), ('Julian Casablancas', 'Last Nite'), ('Alex Turner', 'My Propeller'), ('Arctic Monkeys', 'My Propeller'), ('Dave Grohl', 'Everlong')]>

In [34]:
print(Artist.objects.annotate(latest_released_song_title=Subquery(
    Song.objects.filter(composers=OuterRef("id")).order_by("-release_date").values("song_title")[:1]
)).values_list("name", "latest_released_song_title").query)

SELECT "music_artist"."name", (SELECT U0."song_title" FROM "music_song" U0 INNER JOIN "music_song_composers" U1 ON (U0."id" = U1."song_id") WHERE U1."artist_id" = "music_artist"."id" ORDER BY U0."release_date" DESC LIMIT 1) AS "latest_released_song_title" FROM "music_artist"
